In [3]:
import os
import pandas as pd

data = []

for folder in os.listdir('evaluations'):
  folder_path = os.path.join('evaluations', folder)
  
  if os.path.isdir(folder_path):

    metrics_path = os.path.join(folder_path, 'metrics.csv')
    metrics_df = pd.read_csv(metrics_path)
    
    metrics_df = metrics_df[~metrics_df['class_name'].str.startswith('Mean')]
    
    split_name = folder.split('-')

    row_data = {
      'model': split_name[0].upper()
    }
    row_data['size'] = split_name[2] if split_name[0] == 'clip' else split_name[1]
    row_data['patch_size'] = (split_name[3] if split_name[0] == 'clip' else split_name[2]).replace('patch', '')
    row_data['res'] = split_name[3] if split_name[0] == 'siglip' else ''
    
    for _, row in metrics_df.iterrows():
      class_name = row['class_name']
      f1_score = row['F1 Score']
      row_data[class_name] = f1_score
    
    data.append(row_data)

result_df = pd.DataFrame(data)
result_df = result_df.sort_values(by=['model', 'size']).reset_index(drop=True)

new_header = pd.MultiIndex.from_tuples(
  [('MODEL', 'Name'), ('MODEL', 'Transf. size'), ('MODEL', 'Patch size'), ('MODEL', 'Resolution')] + [('F1 Score', col) for col in result_df.columns[4:]]
)
result_df.columns = new_header
result_df.to_csv('cross_eval_f1.csv', index=False)
result_df



MODEL                                            F1 Score  \
     Name Transf. size Patch size Resolution ANTHONY OF PADUA   
0    CLIP         base         16                       8.70%   
1    CLIP         base         32                       6.20%   
2    CLIP        large         14                      11.38%   
3  SIGLIP         base         16        512           14.74%   
4  SIGLIP        large         16        384           16.49%   
5  SIGLIP       so400m         14        384            7.04%   

                                                                            \
  FRANCIS OF ASSISI  JEROME JOHN THE BAPTIST MARY MAGDALENE   PAUL   PETER   
0            39.74%   0.00%           35.48%         24.35%  0.00%   0.00%   
1            31.09%   0.00%           19.30%         23.35%  0.00%   0.00%   
2            63.51%   1.67%           27.27%         30.40%  6.35%   1.64%   
3            43.43%  16.11%           36.24%         38.94%  0.00%   1.65%   
4            67.02%  22.07%           38.57%         55.33%  0.00%  47.18%   
5            16.51%  27.03%           46.11%         57.64%  0.00%   0.00%   

                                             
  SAINT DOMINIC SAINT SEBASTIAN VIRGIN MARY  
0        14.73%          38.82%      29.90%  
1        10.35%          31.25%      11.42%  
2        15.43%          58.68%      32.98%  
3        11.60%          55.62%      59.35%  
4        18.12%          67.59%      72.98%  
5        21.40%          45.08%      71.48%

In [4]:
import os
import pandas as pd

data = []

for folder in os.listdir('evaluations'):
  folder_path = os.path.join('evaluations', folder)
  
  if os.path.isdir(folder_path):
    metrics_path = os.path.join(folder_path, 'metrics.csv')
    metrics_df = pd.read_csv(metrics_path)
    
    split_name = folder.split('-')
    
    row_data = {
      'model': split_name[0].upper()
    }
    row_data['size'] = split_name[2] if split_name[0] == 'clip' else split_name[1]
    row_data['patch_size'] = (split_name[3] if split_name[0] == 'clip' else split_name[2]).replace('patch', '')
    row_data['res'] = split_name[3] if split_name[0] == 'siglip' else ''
    
    row_data['macro'] = metrics_df.loc[metrics_df['class_name'] == "Mean", 'Average Precision'].values[0]
    row_data['micro'] = metrics_df.loc[metrics_df['class_name'] == "Mean (micro)", 'Average Precision'].values[0]
    
    data.append(row_data)

result_df = pd.DataFrame(data)
result_df = result_df.sort_values(by=['model', 'size']).reset_index(drop=True)

new_header = pd.MultiIndex.from_tuples(
  [('MODEL', 'Name'), ('MODEL', 'Transf. size'), ('MODEL', 'Patch size'), ('MODEL', 'Resolution')] + [('mAP', 'macro'), ('mAP', 'micro')]
)
result_df.columns = new_header
result_df.to_csv('cross_eval_micro-macro.csv', index=False)
result_df

MODEL                                        mAP        
     Name Transf. size Patch size Resolution   macro   micro
0    CLIP         base         16             34.61%  23.14%
1    CLIP         base         32             34.79%  20.74%
2    CLIP        large         14             49.06%  27.26%
3  SIGLIP         base         16        512  48.73%  34.80%
4  SIGLIP        large         16        384  58.47%  39.21%
5  SIGLIP       so400m         14        384  62.22%  37.48%